<center>
<hr>
<h1>Complessità nei sistemi sociali</h1>
<h2>Laurea Magistrale in Fisica Dei Sistemi Complessi</h2>
<h2>A.A. 2018/19</h2>
<h3>Daniela Paolotti & Michele Tizzoni</h3>
<h3>Notebook 4 - Fitting power law distributions</h3>
<hr>
</center>

In [ ]:
import networkx as nx
import seaborn as sns

In [ ]:
import numpy as np
from operator import itemgetter

In [ ]:
%pylab inline

### The Powerlaw package
We use the Python toolbox [powerlaw](https://github.com/jeffalstott/powerlaw) that implements a method proposed by Aaron Clauset and collaborators in [this paper](https://arxiv.org/abs/0706.1062).
The paper explains why fitting a power law distribution using a linear regression of logarthim is not correct. 
A more sound approach is based on a Maximum Likelihood Estimator.

The package can be installed using `pip` as `pip install powerlaw`.
Full documentation is available [here](http://pythonhosted.org/powerlaw/).
Several examples and a detailed description of the library has been published in a paper on [PLOS ONE
](http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0085777).

As stated by Clauset, Shalizi and Newman: 
>In practice, we can rarely, if ever, be certain that an observed quantity is drawn from a power-law distribution. The most we can say is that our observations are consistent with the hypothesis that $x$ is drawn from a distribution of the form of $p(x) \propto x^{-\alpha}$. In some cases we may also be able to rule out some other competing hypotheses.

In [ ]:
import powerlaw as pwl

# Analysis of ca-AstroPh

We analyze the network file 'ca-AstroPh' from the SNAP repository. 
This is a co-authorhip network, thus, it is undirected.

In [ ]:
filepath='./../network_data/ca-AstroPh.txt'

In [ ]:
G=nx.Graph()

fh=open(filepath,'r')
for line in fh.readlines():
    s=line.strip().split()
    if s[0]!='#':
        origin=int(s[0])
        dest=int(s[1])
        G.add_edge(origin,dest)
fh.close()

In [ ]:
print('The network has', len(G), 'nodes and', len(G.edges()), 'links.')

### Degree distribution
Let's plot the degree distribution of the network.

It's important to keep in mind the difference between [probability density function](https://en.wikipedia.org/wiki/Probability_density_function) and [probablity mass function](https://en.wikipedia.org/wiki/Probability_mass_function).

In [ ]:
from collections import Counter 
deg=dict(G.degree()).values()
deg_distri=Counter(deg)

We plot the degree probability mass function.

In [ ]:
x=[]
y=[]
for i in sorted(deg_distri):   
    x.append(i)
    y.append(deg_distri[i]/len(G))

plt.figure(figsize=(10,7))    
plt.plot(x,y)

plt.xlabel('degree $k$', fontsize=18)
plt.ylabel('$P(k)$', fontsize=18)

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.yscale('log')
plt.xscale('log')
plt.axis([1,1000,0.00001,0.2])
plt.show()

Using the 'hist()' function of matplotlib we can plot the probability density distribution, choosing the number of bins.

In [ ]:
plt.figure(figsize=(10,7))
plt.hist(deg, bins=90, normed=True, log=True, histtype='stepfilled')
plt.plot(x,y,'ro')
plt.xscale('log')
plt.yscale('log')
plt.xticks(fontsize=20)
plt.yticks(fontsize=22)
plt.xlabel('degree $k$', fontsize=22)
plt.ylabel('$P(k)$', fontsize=22)

The *powerlaw* package provides direct access to the probability density function.

In [ ]:
degree=list(deg)

In [ ]:
pwl_distri=pwl.pdf(degree, bins=90)

In [ ]:
pwl_distri

In [ ]:
plt.figure(figsize=(10,7))
plt.yscale('log')
plt.xscale('log')
plt.plot(x,y,'ro')

pwl.plot_pdf(degree, color='black', linewidth=2)

plt.xticks(fontsize=20)
plt.yticks(fontsize=22)

plt.xlabel('degree $k$', fontsize=22)
plt.ylabel('$P(k)$', fontsize=22)

In [ ]:
plt.figure(figsize=(10,7))
plt.yscale('log')
plt.xscale('log')
plt.plot(x,y,'ro')

pwl.plot_pdf(degree, linear_bins=True, color='black', linewidth=2)

plt.xticks(fontsize=20)
plt.yticks(fontsize=22)

plt.xlabel('degree $k$', fontsize=22)
plt.ylabel('$P(k)$', fontsize=22)

## Parameter estimation

The library powerlaw allows to estimate the exponent $\alpha$ and the minimum value for the scaling $x_{min}$

In [ ]:
fit_function = pwl.Fit(degree)

In [ ]:
fit_function

In [ ]:
fit_function.power_law

In [ ]:
fit_function.power_law.alpha

In [ ]:
fit_function.power_law.sigma

In [ ]:
fit_function.power_law.xmin

We fix the minimum value for the scaling $x_{min}=10$

In [ ]:
fit_function_fixmin = pwl.Fit(degree, xmin=10)

In [ ]:
fit_function_fixmin.xmin

In [ ]:
fit_function_fixmin.power_law.alpha

In [ ]:
fit_function_fixmin.power_law.sigma

We can look at the values of the [Kolgomorov-Sminorv distance](https://en.wikipedia.org/wiki/Kolmogorov%E2%80%93Smirnov_test) of the two fits to compare them. Smaller distances correspond to better fits.

In [ ]:
fit_function.power_law.D

In [ ]:
fit_function_fixmin.power_law.D

## Visualize distributions and fit

In [ ]:
fig=plt.figure(figsize=(10,7))


plt.plot(x,y,'ro')

fig=pwl.plot_pdf([x for x in degree if x>123], color='r', linewidth=2, label='data')

fit_function.power_law.plot_pdf(ax=fig, color='b', linestyle='-', linewidth=1, label='fit')

fig.legend(fontsize=22)

plt.xticks(fontsize=20)
plt.yticks(fontsize=22)
plt.xlabel('degree $k$', fontsize=22)
plt.ylabel('$P(k)$', fontsize=22)

In [ ]:
fig=plt.figure(figsize=(10,7))

plt.plot(x,y,'ro')

fig=pwl.plot_pdf([x for x in degree if x>10], color='r', linewidth=2, label='Data')

fit_function_fixmin.power_law.plot_pdf(ax=fig, color='b', linestyle='-', linewidth=1, label='Fit')

fig.legend(fontsize=22)
plt.xticks(fontsize=20)
plt.yticks(fontsize=22)
plt.xlabel('Degree', fontsize=22)
plt.ylabel('$P(k)$', fontsize=22)

## Comparing Candidate Distributions

We can compare the goodness of fit of several distributions. Distributions other than a power-law can provide a better fit to the data.

In [ ]:
fit_function.supported_distributions

In [ ]:
R,p = fit_function.distribution_compare('power_law', 'exponential', normalized_ratio=True)

In [ ]:
R,p

R is the loglikelihood ratio between the two candidate distributions. This number will be positive if the data is more likely in the first distribution, and negative if the data is more likely in the second distribution. The significance value for that direction is p. 

In [ ]:
R2,p2 = fit_function.distribution_compare('power_law', 'lognormal_positive', normalized_ratio=True)

In [ ]:
R2,p2

In [ ]:
R3,p3 = fit_function.distribution_compare('power_law', 'truncated_power_law', normalized_ratio=True)

In [ ]:
R3,p3

In [ ]:
R4,p4 = fit_function.distribution_compare('power_law', 'stretched_exponential', normalized_ratio=True)

In [ ]:
R4,p4

Analyze the power law with $x_{min}=10$.

Here, the truncated power law is the best fit that explains the data. Even an exponential is a better fit to the data.

In [ ]:
R,p = fit_function_fixmin.distribution_compare('power_law', 'exponential', normalized_ratio=True)

In [ ]:
R,p

In [ ]:
fig=plt.figure(figsize=(10,7))

fig=pwl.plot_pdf([x for x in degree if x>10], color='r', linewidth=2, label='Data')

fit_function_fixmin.exponential.plot_pdf(ax=fig, color='black', linestyle='-', linewidth=2, label='Fit')

fig.legend(fontsize=22)

plt.xticks(fontsize=20)
plt.yticks(fontsize=22)
plt.xlabel('degree $k$', fontsize=22)
plt.ylabel('$P(k)$', fontsize=22)

In [ ]:
R3, p3 = fit_function_fixmin.distribution_compare('power_law', 'truncated_power_law', normalized_ratio=True)
R3, p3

In [ ]:
fig=plt.figure(figsize=(10,7))

fig=pwl.plot_pdf([x for x in degree if x>10], color='r', linewidth=2, label='Data')

fit_function_fixmin.truncated_power_law.plot_pdf(ax=fig, color='black', linestyle='-', linewidth=2, label='Fit')

fig.legend(fontsize=22)
plt.xticks(fontsize=20)
plt.yticks(fontsize=22)
plt.xlabel('Degree', fontsize=22)
plt.ylabel('$P(k)$', fontsize=22)